<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_sentence_embed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#질답세션의 임베딩 벡터를 추가한 뒤 .csv로 저장하는 코드(로컬)

In [ ]:
!pip install -U sentence-transformers #서버 업로드 시 삭제
!pip install datasets

import numpy as np
import os
import pandas as pd
import urllib.request
import time
import torch
import json
import torch.nn as nn
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from google.colab import drive
from transformers import AutoModel, AutoTokenizer, shape_list, TFBertModel, RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline, pipeline, BertTokenizer, BertForNextSentencePrediction,  TrainingArguments, BertForMaskedLM, Trainer, TrainerCallback
from datasets import Dataset, load_dataset
from sklearn.model_selection import StratifiedKFold, train_test_split
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
#.csv파일 불러오기
file_path = '/content/drive/My Drive/summary_result_short.csv'
df = pd.read_csv(file_path)

#데이터프레임의 question 열을 리스트로 변형
question_data = df['question'].to_list()
answer_data = df['answer'].to_list()

In [ ]:
print(len(question_data))
print(len(answer_data))

3165
3165


In [ ]:
# #모델 불러오기
# model_path = '/content/drive/My Drive/Pretrained_Model_sentence_embed'

# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModel.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of BertModel were not initialized from the model checkpoint at /content/drive/My Drive/Pretrained_Model_sentence_embed and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(43041, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
#모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HUGGINGFACE_MODEL_PATH = 'BM-K/KoSimCSE-bert-multitask'

model = AutoModel.from_pretrained(HUGGINGFACE_MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/675 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
# #수동 배치처리
# batch_size = 64

# result = []
# for i in tqdm(range(0, len(data), batch_size), desc = "Embedding"):
#     batch_texts = data[i : i + batch_size]
#     inputs = tokenizer(batch_texts, padding = True, truncation = True, return_tensors = 'pt')
#     inputs = {k : v.to(device) for k, v in inputs.items()}
#     #모델 실행
#     with torch.no_grad():
#         outputs = model(**inputs)

#     mean_pooling = torch.mean(outputs.last_hidden_state, dim = 1)
#     result.append(mean_pooling)

def get_embeddings(batch_size, data):
  result = []
  for i in tqdm(range(0, len(data), batch_size), desc = "Embedding"):
      batch_texts = data[i : i + batch_size]
      inputs = tokenizer(batch_texts, padding = True, truncation = True, return_tensors = 'pt')
      inputs = {k : v.to(device) for k, v in inputs.items()}
      #모델 실행
      with torch.no_grad():
          outputs = model(**inputs)

      mean_pooling = torch.mean(outputs.last_hidden_state, dim = 1)
      result.append(mean_pooling)

  return result

In [ ]:
# print(len(result))
# print(len(result[0]))
# print(len(result[0][0]))

In [ ]:
# print(len(qresult))
# print(sum(len(lists) for lists in qresult))
# print(len(aresult))
# print(len(df))

50
3165
50
3165


In [ ]:
#데이터프레임에 새로운 열로 추가
qresult = get_embeddings(64, question_data)
aresult = get_embeddings(64, answer_data)
df['question_embedding'] = [sentence.cpu().numpy() for batches in qresult for sentence in batches]
df['answer_embedding'] = [sentence.cpu().numpy() for batches in aresult for sentence in batches]

#비율을 적용한 벡터합
k = 0.8
df['embedding'] = [k * df.iloc[i]['question_embedding'] + (1 - k) * df.iloc[i]['answer_embedding'] for i in range(len(df))]

#csv파일로 저장할 때 쉼표가 사라지지 않게 해야 함, json.dumps 사용
df['embedding'] = df['embedding'].apply(lambda x: json.dumps(x.tolist()))

df = df.drop(['question_embedding', 'answer_embedding'], axis = 1)

Embedding: 100%|██████████| 13/13 [00:21<00:00,  1.68s/it]


In [ ]:
print(qresult[:3])
print(aresult[:3])
print(df['embedding'].to_list()[:3])

In [ ]:
print(len(df['embedding'][55]))
print(df['embedding'])

16096
0       [-0.15344443917274475, -0.2576753497123718, 0....
1       [-0.34569668769836426, -0.8086515665054321, 0....
2       [-0.3544427156448364, -0.5906268954277039, 0.4...
3       [0.5383281111717224, -0.9759604930877686, -0.0...
4       [-0.2234751582145691, -0.9337076544761658, 0.2...
                              ...                        
3160    [-0.15476760268211365, -0.9572567939758301, 0....
3161    [0.15289956331253052, -1.0896754264831543, 0.0...
3162    [0.16936582326889038, -0.9480101466178894, 0.4...
3163    [-0.49828675389289856, -0.7064148783683777, -0...
3164    [-0.16725221276283264, -0.5694186091423035, 0....
Name: embedding, Length: 3165, dtype: object


In [ ]:
#.csv 파일로 google drive에 저장 (나중에 google cloud storage에 저장해야 함)
save_path = '/content/drive/My Drive/sentence_embed_result_short.csv'

df.to_csv(save_path)